In [32]:
# import
import pandas as pd

# remove warnings
import warnings
warnings.filterwarnings("ignore")


# enable autoreload to automatically reload modified modules during development
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocessing of the Data

Before beginning the analysis, it's essential to construct a robust dataset that serves as a reliable foundation for our exploration and insights.

## CMU Movie Summary Corpus

The CMU Movie Summary Corpus serves as our primary dataset. It can be accessed through the following website: [CMU Movie Summary Corpus](https://www.cs.cmu.edu/~ark/personas/). Please note that MovieSummaries isn't pushed on our repo due to size issues. 


In [23]:
# define file paths for the datasets
plot_summaries_path = r'..\MovieSummaries\plot_summaries.txt'
movie_metadata_path = r'..\MovieSummaries\movie.metadata.tsv'
character_metadata_path = r'..\MovieSummaries\character.metadata.tsv'

# load plot summaries with specified column names
plot_summaries = pd.read_csv(
    plot_summaries_path, 
    sep='\t', 
    header=None, 
    names=['Wikipedia_ID', 'Plot_Summary']
)

# load movie metadata with specified column names
movie_metadata = pd.read_csv(
    movie_metadata_path, 
    sep='\t', 
    header=None, 
    names=[
        'Wikipedia_ID', 'Freebase_ID', 'Movie_Name', 'Release_Date', 
        'Box_Office_Revenue', 'Runtime', 'Languages', 'Countries', 'Genres'
    ]
)

# load character metadata with specified column names
character_metadata = pd.read_csv(
    character_metadata_path, 
    sep='\t', 
    header=None, 
    names=[
        'Wikipedia_ID', 'Freebase_ID', 'Release_Date', 'Character_Name', 
        'Actor_Birth_Date', 'Actor_Gender', 'Actor_Height', 'Actor_Ethnicity', 
        'Actor_Name', 'Actor_Age', 'Character_Actor_Freebase_ID', 
        'Character_Freebase_ID', 'Actor_Freebase_ID'
    ]
)

# display a preview of the movie metadata and its dimensions
display(movie_metadata.head(2), movie_metadata.shape)


,Wikipedia_ID,Freebase_ID,Movie_Name,Release_Date,Box_Office_Revenue,Runtime,Languages,Countries,Genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."


(81741, 9)

### Treatment colomns Languages, Countries and Genres
The code loads a CSV file with movie metadata into a DataFrame, defining names for each column. The "Freebase_ID" column is removed, and the "Languages," "Countries," and "Genres" columns are converted into lists of values by extracting data from dictionary-like strings. Empty cells are replaced with "Unknown." Finally, the first few rows of the DataFrame are displayed for verification.

In [24]:
# load movie metadata with specified columns
movie_metadata = pd.read_csv(
    movie_metadata_path, 
    sep='\t', 
    header=None, 
    names=[
        'Wikipedia_ID', 'Freebase_ID', 'Movie_Name', 'Release_Date', 
        'Box_Office_Revenue', 'Runtime', 'Languages', 'Countries', 'Genres'
    ]
)

# drop the 'Freebase_ID' column as it's not needed for further analysis
movie_metadata = movie_metadata.drop(columns=['Freebase_ID'], inplace=False)

# explode the 'Languages' column into a list of values, defaulting to ['Unknown'] if empty
movie_metadata['Languages'] = movie_metadata['Languages'].apply(
    lambda x: list(eval(x).values()) if x != '{}' else ['Unknown']
)

# explode the 'Countries' column into a list of values, defaulting to ['Unknown'] if empty
movie_metadata['Countries'] = movie_metadata['Countries'].apply(
    lambda x: list(eval(x).values()) if x != '{}' else ['Unknown']
)

# explode the 'Genres' column into a list of values, defaulting to ['Unknown'] if empty
movie_metadata['Genres'] = movie_metadata['Genres'].apply(
    lambda x: list(eval(x).values()) if x != '{}' else ['Unknown']
)


### Mapping Countries to Continents and Filtering for America and Europe

This code defines a dictionary to classify countries by continent, then uses a function to map each country to its respective continent. If a country does not match any defined continent, it is labeled as "Other." This mapping function is applied to the "Countries" column in the `movie_metadata` DataFrame to create a new column, "Continents." Finally, the DataFrame is filtered to retain only rows where the continent includes "America" or "Europe".

In [25]:
from src.utils.data_utils import map_country_list_to_continent

# apply the mapping function to the 'Countries' column to determine the corresponding continents
movie_metadata['Continents'] = movie_metadata['Countries'].apply(
    lambda x: map_country_list_to_continent(x)
)

# filter rows where the 'Continents' column contains either 'America' or 'Europe'
movie_metadata = movie_metadata[movie_metadata['Continents'].apply(
    lambda x: 'America' in x or 'Europe' in x
)]

In [26]:
from src.utils.data_utils import treat_continent

# apply the 'treat_continent' function to standardize the 'Continents' column
movie_metadata["Continents"] = movie_metadata["Continents"].apply(lambda x: treat_continent(x))

# filter out rows where the 'Continents' list contains 3 or more elements
movie_metadata = movie_metadata[movie_metadata["Continents"].apply(lambda x: len(x) < 3)]

# display the updated value counts for the 'Continents' column to inspect the distribution
display(movie_metadata["Continents"].value_counts())

Continents
[America]             34779
[Europe]              18488
[Both]                 3000
[America, Asia]         456
[Europe, Asia]          366
[Europe, Africa]        195
[America, Oceania]      182
[Both, Asia]            130
[Europe, Oceania]        93
[America, Africa]        63
[Both, Oceania]          50
[Both, Africa]           35
Name: count, dtype: int64

### Filtering for Comedy Genre and Counting Occurrences

This code filters the movies to keep only those with the genre "Comedy." It sets other genres to `None`, then drops rows without "Comedy" in the "Genres" column. 

In [27]:
# filter the 'Genres' column, keeping only rows where 'Comedy' is in the genre list
movie_metadata["Genres"] = movie_metadata["Genres"].apply(lambda x: x if 'Comedy' in x else None)

# remove rows where 'Genres' is NaN (i.e., not containing 'Comedy')
movie_metadata = movie_metadata.dropna(subset=["Genres"])

# display the value counts for the 'Genres' column to inspect the distribution
movie_metadata["Genres"].value_counts()

Genres
[Comedy]                                                                           1700
[Short Film, Silent film, Indie, Black-and-white, Comedy]                           547
[Drama, Comedy]                                                                     358
[Short Film, Comedy, Black-and-white]                                               252
[Comedy, Black-and-white]                                                           250
                                                                                   ... 
[Silent film, Melodrama, Indie, Comedy-drama, Black-and-white, Comedy]                1
[LGBT, Gay Interest, Indie, Gay, Satire, Comedy, Gay Themed, Romance Film]            1
[LGBT, Horror, Comedy-drama, Psychological thriller, Comedy, Black comedy]            1
[Period piece, Drama, Comedy, Film adaptation, Black comedy, Biographical film]       1
[Crime Fiction, Thriller, Comedy, Supernatural]                                       1
Name: count, Length: 6092

### Filling Missing Box Office Values and Verifying Changes


This code fills any `NaN` values in the "Box_Office_Revenue" column with 0. Then, it displays the first few rows and the shape of the DataFrame to verify the changes.

In [28]:
# fill the NaN values in the Box_Office_Revenue column with 0
movie_metadata['Box_Office_Revenue'] = movie_metadata['Box_Office_Revenue'].fillna(0)

## Open Movie Database (OMBD)

In [29]:
# load the OMDB API data from the specified CSV file
# note: this dataset is not included in the repository due to size constraints
omdb_data_path = r'..\MovieSummaries\movies_omdbapi.csv'
omdb_data = pd.read_csv(omdb_data_path)

# display the first few rows of the dataframe and its shape to inspect the data
display(omdb_data.head(2), omdb_data.shape)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,Ghosts of Mars,2001,R,24 Aug 2001,98 min,"Action, Horror, Sci-Fi",John Carpenter,"Larry Sulkis, John Carpenter","Natasha Henstridge, Ice Cube, Pam Grier","In 2176, a Martian police unit is sent to pick...",...,4.9,"58,722",tt0228333,movie,NaN,"$8,709,640",NaN,NaN,True,NaN
1,Getting Away with Murder: The JonBenet Ramsey ...,2000,NaN,16 Feb 2000,95 min,Drama,Edward Lucas,"Michael A. Graham, Ted Haimes","Alice Barrett, Rod Britt, Hildy Brooks","Six-year-old JonBenét Ramsey, whose body was f...",...,6.0,69,tt0245916,movie,NaN,NaN,NaN,NaN,True,NaN


(71544, 26)

In [30]:
# let's inspect the columns of the OMDB dataset
print(omdb_data.columns)

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response',
       'totalSeasons'],
      dtype='object')


## Merging DataFrames and Dropping Unnecessary Columns

This code merges two DataFrames, movie_metadata and omdb_data, using an inner join on the "Movie_Name" column from movie_metadata and the "Title" column from omdb_data.Afterward, it removes unneeded columns ("Released," "Genre," "Title," "Language," "Country," and "Type") to keep only relevant information.

In [31]:
# merge the two DataFrames on the 'Movie_Name' column from 'movie_metadata' and 'Title' column from 'omdb_data'
# we use an inner join to keep only matching rows between the two datasets
merged_movie_metadata = pd.merge(
    movie_metadata, 
    omdb_data, 
    left_on='Movie_Name', 
    right_on='Title', 
    how='inner'
)

# drop unnecessary columns from the merged DataFrame
merged_movie_metadata = merged_movie_metadata.drop(columns=[
    "Released", "Genre", "Title", "Language", "Country", "Type"
])

### Cleaning and Merging Box Office Revenue Data

This code combines revenue data from "Box_Office_Revenue" and "BoxOffice" to ensure complete, consistent values. It removes the dollar sign from "BoxOffice," converts it to a float, replaces 0s in "Box_Office_Revenue" with `NaN`, and fills missing values from "BoxOffice." The "BoxOffice" column is then dropped, leaving a unified "Box_Office_Revenue" column in float format. This process standardizes revenue data for further analysis.

In [33]:
# display the initial state of 'Box_Office_Revenue' and 'BoxOffice' columns
print("Initial 'Box_Office_Revenue' and 'BoxOffice' columns:")
print(merged_movie_metadata[["Box_Office_Revenue", "BoxOffice"]].head())

# clean and convert 'BoxOffice' column: remove dollar signs and commas, then convert to float
merged_movie_metadata['BoxOffice'] = merged_movie_metadata['BoxOffice'].replace(r'[\$,]', '', regex=True).astype(float)

# replace 0 values in 'Box_Office_Revenue' with NaN
merged_movie_metadata['Box_Office_Revenue'] = merged_movie_metadata['Box_Office_Revenue'].replace(0, pd.NA)

# merge 'BoxOffice' into 'Box_Office_Revenue': fill missing values in 'Box_Office_Revenue' with 'BoxOffice' values
merged_movie_metadata['Box_Office_Revenue'] = (
    merged_movie_metadata['Box_Office_Revenue']
    .fillna(merged_movie_metadata['BoxOffice'])
    .infer_objects()
)

# drop the redundant 'BoxOffice' column
merged_movie_metadata = merged_movie_metadata.drop(columns=['BoxOffice'])

# display the updated 'Box_Office_Revenue' column
print("\nUpdated 'Box_Office_Revenue' column:")
print(merged_movie_metadata[['Box_Office_Revenue']].head())

# display the number of missing values in 'Box_Office_Revenue'
print(f"\n------------------------------------")
missing_values = merged_movie_metadata['Box_Office_Revenue'].isna().sum()
print(f"Number of missing values in 'Box_Office_Revenue': {missing_values}")

# verify if all values in 'Box_Office_Revenue' are floats
print(f"\n------------------------------------")
is_float = merged_movie_metadata['Box_Office_Revenue'].apply(lambda x: isinstance(x, float)).all()
print(f"All values in 'Box_Office_Revenue' are floats: {is_float}")


Initial 'Box_Office_Revenue' and 'BoxOffice' columns:
   Box_Office_Revenue     BoxOffice
0                 0.0           NaN
1           3600000.0           NaN
2                 0.0           NaN
3         102272727.0  $102,272,727
4                 0.0       $69,171

Updated 'Box_Office_Revenue' column:
   Box_Office_Revenue
0                 NaN
1           3600000.0
2                 NaN
3         102272727.0
4             69171.0

------------------------------------
Number of missing values in 'Box_Office_Revenue': 8429

------------------------------------
All values in 'Box_Office_Revenue' are floats: True


### Cleaning and Merging Runtime Data

This code standardizes the runtime data by removing unwanted characters and combining columns. First, digits are removed from "Runtime_y" to identify common patterns, and rows with invalid characters ('h' or 'S') are filtered out. The column is further cleaned by stripping "min" and converting values to float. Any missing values in "Runtime_x" are then filled using "Runtime_y," after which "Runtime_y" is dropped. Finally, "Runtime_x" is renamed to "Runtime" for clarity, ensuring all runtime data is consolidated and ready for analysis.

In [34]:
# remove digits from the 'Runtime_y' column and count occurrences of the cleaned values
runtime_counts = (
    merged_movie_metadata['Runtime_y']
    .str.replace(r'\d+', '', regex=True)  # remove digits
    .value_counts()  # count occurrences of each unique value
)

# display the count of cleaned runtime values
print("Counts of runtime values after removing digits:")
print(runtime_counts)

# filter out rows where 'Runtime_y' contains 'h' or 'S', indicating irrelevant values
merged_movie_metadata = merged_movie_metadata[
    ~merged_movie_metadata['Runtime_y'].str.contains(r'h|S', na=False)  # exclude rows with 'h' or 'S'
]

# display the first few rows of the updated 'Runtime_y' column after filtering
print("\nFirst few rows of 'Runtime_y' after filtering:")
print(merged_movie_metadata[['Runtime_y']].head())

# remove digits from the 'Runtime_y' column again and count occurrences
runtime_counts = (
    merged_movie_metadata['Runtime_y']
    .str.replace(r'\d+', '', regex=True)  # remove digits
    .value_counts()  # count occurrences of each unique value
)

# display the final count of cleaned runtime values
print("\nFinal counts of runtime values after filtering:")
print(runtime_counts)

Counts of runtime values after removing digits:
Runtime_y
 min       13330
S min         31
 h  min        5
 h             1
Name: count, dtype: int64

First few rows of 'Runtime_y' after filtering:
  Runtime_y
0   105 min
1   106 min
2    70 min
3   139 min
4    83 min

Final counts of runtime values after filtering:
Runtime_y
min    13330
Name: count, dtype: int64


In [35]:
# merge runtime columns: clean 'Runtime_y' and update 'Runtime_x' with 'Runtime_y' values

# remove 'min' from 'Runtime_y' and convert to float
merged_movie_metadata['Runtime_y'] = merged_movie_metadata['Runtime_y'].str.replace(' min', '').astype(float)

# replace NaN values in 'Runtime_x' with values from 'Runtime_y'
merged_movie_metadata['Runtime_x'].fillna(merged_movie_metadata['Runtime_y'], inplace=True)

# drop the 'Runtime_y' column as it is now merged
merged_movie_metadata = merged_movie_metadata.drop(columns=['Runtime_y'])

# rename 'Runtime_x' to 'Runtime' for clarity
merged_movie_metadata.rename(columns={'Runtime_x': 'Runtime'}, inplace=True)

### Extracting and Organizing Ratings Data


This code extracts individual ratings from a JSON-like "Ratings" column. It defines a function to parse `ratings_str`, converting it to a dictionary of ratings by source (e.g., IMDb, Rotten Tomatoes). This function is applied to create a "Ratings_Dict" column, from which specific ratings are extracted into separate columns: "Internet_Movie_Database_Rating," "Rotten_Tomatoes_Rating," and "Metacritic_Rating." 

In [36]:
from src.utils.data_utils import extract_awards

# apply the extract_awards function to the 'Awards' column and create new columns for each award category
merged_movie_metadata[['Oscar', 'Nomination_Awards', 'Win_Awards']] = merged_movie_metadata['Awards'].apply(
    lambda x: pd.Series(extract_awards(x))
)

### Extracting, Normalizing, and Verifying Ratings Data



This code extracts and standardizes movie ratings to a 10-point scale. Initially, it parses the "Ratings" column to create a dictionary of ratings by source (IMDb, Rotten Tomatoes, Metacritic). Each rating source is extracted into separate columns, and a function then normalizes the values to a scale of 10, converting percentages and different scales as needed. The transformed columns are verified to ensure all values are floats and below 10, confirming the consistency of the rating data for analysis.

In [38]:
from src.utils.data_utils import extract_ratings

# apply the extract_ratings function to the 'Ratings' column and store the result in a new 'Ratings_Dict' column
merged_movie_metadata['Ratings_Dict'] = merged_movie_metadata['Ratings'].apply(extract_ratings)

# extract ratings from different sources and store them in separate columns
merged_movie_metadata['Internet_Movie_Database_Rating'] = merged_movie_metadata['Ratings_Dict'].apply(
    lambda x: x.get('Internet Movie Database', None)
)
merged_movie_metadata['Rotten_Tomatoes_Rating'] = merged_movie_metadata['Ratings_Dict'].apply(
    lambda x: x.get('Rotten Tomatoes', None)
)
merged_movie_metadata['Metacritic_Rating'] = merged_movie_metadata['Ratings_Dict'].apply(
    lambda x: x.get('Metacritic', None)
)

# drop the temporary 'Ratings_Dict' column as it is no longer needed
merged_movie_metadata = merged_movie_metadata.drop(columns=['Ratings_Dict'])

# display the first few rows of relevant columns to verify the changes
display(merged_movie_metadata[['Movie_Name', 'Internet_Movie_Database_Rating', 'Rotten_Tomatoes_Rating', 'Metacritic_Rating']].head())

,Movie_Name,Internet_Movie_Database_Rating,Rotten_Tomatoes_Rating,Metacritic_Rating
0,The Gangsters,5.9/10,None,None
1,Alexander's Ragtime Band,6.8/10,83%,None
2,Contigo y aquí,None,None,None
3,Mary Poppins,7.8/10,97%,88/100
4,White on Rice,6.1/10,45%,45/100


In [39]:
from src.utils.data_utils import convert_to_scale_of_10

# apply the 'convert_to_scale_of_10' function to the rating columns to standardize them
merged_movie_metadata['Internet_Movie_Database_Rating'] = merged_movie_metadata['Internet_Movie_Database_Rating'].apply(convert_to_scale_of_10)
merged_movie_metadata['Rotten_Tomatoes_Rating'] = merged_movie_metadata['Rotten_Tomatoes_Rating'].apply(convert_to_scale_of_10)
merged_movie_metadata['Metacritic_Rating'] = merged_movie_metadata['Metacritic_Rating'].apply(convert_to_scale_of_10)

In [40]:
# scale the 'Metacritic_Rating' column to a 0-1 scale by dividing by 100
merged_movie_metadata['Metacritic_Rating'] = merged_movie_metadata['Metacritic_Rating'] / 100

# display the first few rows of the relevant columns to verify the changes
display(merged_movie_metadata[['Movie_Name', 'Internet_Movie_Database_Rating', 'Rotten_Tomatoes_Rating', 'Metacritic_Rating']].head())

,Movie_Name,Internet_Movie_Database_Rating,Rotten_Tomatoes_Rating,Metacritic_Rating
0,The Gangsters,5.9,NaN,NaN
1,Alexander's Ragtime Band,6.8,8.3,NaN
2,Contigo y aquí,NaN,NaN,NaN
3,Mary Poppins,7.8,9.7,8.8
4,White on Rice,6.1,4.5,4.5


In [41]:
# verify that the specified columns contain only numbers less than 10 and are floats
columns_to_check = ['Internet_Movie_Database_Rating', 'Rotten_Tomatoes_Rating', 'Metacritic_Rating']

# loop through each column to perform the checks
for column in columns_to_check:
    # check if all values are less than or equal to 10
    all_less_than_10 = merged_movie_metadata[column].apply(
        lambda x: x <= 10 if pd.notna(x) else True
    ).all()

    # check if all values are floats
    all_floats = merged_movie_metadata[column].apply(
        lambda x: isinstance(x, float) if pd.notna(x) else True
    ).all()

    # print the results of the checks for each column
    print(f"All values in '{column}' are less than 10: {all_less_than_10}")
    print(f"All values in '{column}' are floats: {all_floats}")

All values in 'Internet_Movie_Database_Rating' are less than 10: True
All values in 'Internet_Movie_Database_Rating' are floats: True
All values in 'Rotten_Tomatoes_Rating' are less than 10: True
All values in 'Rotten_Tomatoes_Rating' are floats: True
All values in 'Metacritic_Rating' are less than 10: True
All values in 'Metacritic_Rating' are floats: True


### Classifying Age Ratings (Rated)

This code classifies movie age ratings into broader categories. It defines a function, classifier_age, which assigns ratings to specific groups: "All Audiences (TP)," "Recommended Parental Agreement (AP)," "13+," "16+," "18+," and "Not Rated." Using predefined lists of ratings, it categorizes each entry in the "Rated" column of merged_movie_metadata.

In [42]:
from src.utils.data_utils import classifier_age

# display the value counts for the 'Rated' column to inspect the distribution
merged_movie_metadata["Rated"].value_counts()

# apply the 'classifier_age' function to the 'Rated' column to categorize the ratings
merged_movie_metadata["Rated"] = merged_movie_metadata["Rated"].apply(classifier_age)

In [21]:
# Save to a CSV file
#merged_movie_metadata.to_csv(merged_movie_metadata.csv, index=False)